# Projeto 3 de Ciência dos Dados

Integrantes:

Ana Clara
João Pedro Varella
João Guilherme

## Imports

In [ ]:
import pandas as pd
import ast
from pprint import pprint
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from sklearn.datasets import load_digits
digits = load_digits()

In [ ]:
dir(digits)

## Lendo arquivos

In [ ]:
df = pd.read_csv("movies_metadata.csv")
df2 = pd.read_csv("credits.csv")
df2.head()

## Limpeza dos dados

### Retirada dos JSONs

No dataset utilizado, algumas variáveis estavam codificadas na forma de JSON. Para acessá-las, foi necessário extrair as informações e transformá-las em um dicionário. Nesta etapa da análise, extraímos os nomes dos atores de cada um dos filmes. Dessa forma, foi possível criar um dicionário relacionando duas variáveis, em que a chave corresponde ao nome do ator, e o valor dessa chave é o número de filmes em que cada ator atuou.

In [ ]:
from collections import defaultdict
atores = defaultdict(int)
for e in df2.index:
    data_dict = ast.literal_eval(df2["cast"][e])
    for palavras in data_dict:
        nome = palavras['name']
        
        atores[nome] += 1

### Código para limpeza de certas variáveis tais como: revenue, budget, produtora e gênero:

In [ ]:
df_0 = df[df['revenue'] > 0]
df_budget = df_0[df_0['budget'] != '0']
df_clean = df_budget[df_budget['production_companies'] != '[]']
df_super_clean = df_clean[df_clean['genres'] != '[]']

In [ ]:
df_mega_clean = df_super_clean[["id","genres","budget","revenue","production_companies"]]
df_mega_clean['budget'] = df_mega_clean['budget'].astype(float)
df_mega_clean.head()

### Transformando o dicionário em DataFrame:

In [ ]:
ser_atores = pd.Series(atores)
ser_atores.head()

Para visualizar de maneira mais ordenada e clara os dados do DataFrame, selecionamos a parcela de 0.1% de atores com mais produções no cinema. Dessa forma, atingimos todos com mais de 28 aparições:

In [ ]:
ser_atores = ser_atores[ser_atores > 28].sort_values(ascending=False)

Para utilizar os dados do DataFrame acima no método do Random Forrest, foi necessário transformar as informações nele em um outro DataFrame com solução binária.

Nesta nova tabela, cada linha representa cada um dos filmes contidos no database, e cada coluna representa os atores selecionados previamente:

In [ ]:
df_filmes_atores = pd.DataFrame(index=df2.index, columns=ser_atores.index, dtype=np.uint8)
for e in df2.index:
    data_dict = ast.literal_eval(df2["cast"][e])
    for palavras in data_dict:
        nome = palavras['name']
        if nome in df_filmes_atores.columns:
            df_filmes_atores[nome][e] = 1
#atores

In [ ]:
df_filmes_atores[df_filmes_atores != 1] = 0
df_filmes_atores.head()

##### Nota-se que neste DataFrame, a grande maioria dos dígitos é 0, o que representa o óbvio, os atores apareceram em poucos filmes em relação ao total estudado

In [ ]:
df.drop_duplicates(subset ="original_title", 
                     keep = 'first', inplace = True) 
df["original_title"].value_counts().value_counts()


In [ ]:
df['bilhao'] = df.revenue > 1e9

#### Código para limpeza de certas variáveis tais como: revenue, budget, produtora e gênero:

In [ ]:
df_0 = df[df['revenue'] > 0]
df_budget = df_0[df_0['budget'] != '0']
df_clean = df_budget[df_budget['production_companies'] != '[]']
df_super_clean = df_clean[df_clean['genres'] != '[]']

In [ ]:
df_mega_clean = df_super_clean[["id","genres","budget","revenue","production_companies"]]
df_mega_clean['budget'] = df_mega_clean['budget'].astype(float)
df_mega_clean.head()

In [ ]:
milhao = []
# df_dinheiro = df_mega_clean[df_mega_clean['revenue'] > 1.5e8]
for dinheiro in df_mega_clean.revenue:
    if dinheiro > 1.5e8:
        milhao.append(1)
    else:
        milhao.append(0)

In [ ]:
df_mega_clean['Fez ou não 150 mi'] = milhao
df_mega_clean.head()

### Selecionando os atores a partir dos filmes filtrados do df_mega_clean

In [ ]:
lista_teste = []
for numero in df_filmes_atores.index:
    if numero not in df_mega_clean.id:
        lista_teste.append(numero)
        
atores_certos = df_filmes_atores.drop(lista_teste, axis=0)

No dataframe acima, foi selecionado os atores que participam dos filmes filtrados no Dataframe "mega_clean" fazendo com que agora podemos observar quais atores participaram dos filmes relevantes 

In [ ]:
atores_certos['Fez ou não 150 mi'] = milhao
atores_certos.head()

In [ ]:
sim_ou_nao = atores_certos['Fez ou não 150 mi']
y = df_mega_clean['Fez ou não 150 mi']

### Testando a relação do ator com a bilheteria do filme

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(atores_certos.drop(['Fez ou não 150 mi'],axis='columns'),sim_ou_nao,test_size = 0.3)

In [ ]:
len(X_test)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators = 40)
model.fit(X_train,Y_train)

In [ ]:
model.score(X_test,Y_test)

A partir dessa análise é visível que os atores não influenciam o filme passar ou não dos 150 milhoes de bilheteria

#### Gráfico de barras relacionando cada gênero com o seu número de filmes

In [ ]:
generos = []
for e in df_mega_clean.index:
    data_dict = ast.literal_eval(df_mega_clean["genres"][e])
    for palavras in data_dict:
        generos.append(palavras['name'])

df_generos = pd.DataFrame(generos)
df_generos
geneross = df_generos[0].value_counts()
geneross
geneross.plot(kind='bar')


### Relação genero com a bilheteria

In [ ]:
revenues_por_categoria = {}
categorias = [
    'Drama', 'Comedy', 'Thriller',
    'Action','Romance','Adventure','Crime','Science Fiction',
    'Horror','Family','Fantasy','Mystery','Animation','History',
    "War",'Music','Western','Documentary','Foreign','TV Movie'
]

for c in categorias:
    revenues_por_categoria[c] = []

for e in df_mega_clean.index:
    data_dict = ast.literal_eval(df_mega_clean["genres"][e])
    for palavras in data_dict:
        cat = palavras["name"]
        #print(cat)
        if cat in revenues_por_categoria:
            revenues_por_categoria[cat].append(df_mega_clean['revenue'][e])
            
for k in revenues_por_categoria:
    revenues_por_categoria[k] = (np.array(revenues_por_categoria[k]))

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
fig = plt.figure(figsize=(16,8))
ax = fig.add_subplot(111)
ax.boxplot(revenues_por_categoria.values(), showfliers=False)
ax.set_xticklabels(revenues_por_categoria.keys(), rotation='vertical');
#plt.yscale('log')
plt.show()


In [ ]:
categorias2 = defaultdict(int)
for e in df_mega_clean.index:
    data_dict = ast.literal_eval(df_mega_clean["genres"][e])
    for palavras in data_dict:
        tipo = palavras['name']
        categorias2[tipo] += 1
categorias2

In [ ]:
ser_categorias = pd.Series(categorias2)

In [ ]:
df_filmes_categorias = pd.DataFrame(index=df_mega_clean.index, columns=ser_categorias.index, dtype=np.uint8)
for e in df_mega_clean.index:
    data_dict = ast.literal_eval(df_mega_clean["genres"][e])
    #print(data_dict)
    for palavras in data_dict:
        tipo = palavras['name']
        #print(tipo)
        if tipo in df_filmes_categorias.columns:
             df_filmes_categorias[tipo][e] = 1


In [ ]:
df_filmes_categorias[df_filmes_categorias != 1] = 0
df_filmes_categorias.head()

In [ ]:
X2_train, X2_test, Y2_train, Y2_test = train_test_split(df_filmes_categorias,sim_ou_nao,test_size = 0.3)

model = RandomForestClassifier(n_estimators = 40)
model.fit(X2_train,Y2_train)

In [ ]:
model.score(X2_test,Y2_test)

#### Gráfico de barras com cada produtora com o número de filmes produzidos por ela:

In [ ]:
produtoras = []
for e in df_mega_clean.index:
    data_dict = ast.literal_eval(df_mega_clean["production_companies"][e])
   # print(data_dict)
    for palavras in data_dict:
        produtoras.append(palavras['name'])

df_produtoras = pd.DataFrame(produtoras)
df_produtoras
produtorass = df_produtoras[0].value_counts().head(30)
produtorass
produtorass.plot(kind='bar')

### Relação produtora com a bilheteria

In [ ]:
revenues_por_produtora = {}
produtoras = [
    'Warner Bros.','Universal Pictures','Paramount Pictures','Twentieth Century Fox Film Corporation','Columbia Pictures',
    'New Line Cinema','Metro-Goldwyn-Mayer (MGM)','Touchstone Pictures','Walt Disney Pictures','Columbia Pictures Corporation',
    'Relativity Media','United Artists','Miramax Films','TriStar Pictures','Canal+','Village Roadshow Pictures','DreamWorks SKG',
    'Regency Enterprises','Lionsgate','Amblin Entertainment','Summit Entertainment','Dune Entertainment','Fox Searchlight Pictures',
    'Dimension Films','Working Title Films','Fox 2000 Pictures','StudioCanal','Silver Pictures','Hollywood Pictures',
    'The Weinstein Company'
]

for p in produtoras:
    revenues_por_produtora[p] = []

for f in df_mega_clean.index:
    data_dict = ast.literal_eval(df_mega_clean["production_companies"][f])
    #print(data_dict)
    for palavras in data_dict:
        #print(palavras)
        prod = palavras["name"]
        if prod in revenues_por_produtora:
            revenues_por_produtora[prod].append(df_mega_clean['revenue'][f])
        
            
#revenues_por_produtora
            
for k in revenues_por_produtora:
    revenues_por_produtora[k] = (np.array(revenues_por_produtora[k]))

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
fig = plt.figure(figsize=(16,8))
ax = fig.add_subplot(111)
ax.boxplot(revenues_por_produtora.values(), showfliers=False)
ax.set_xticklabels(revenues_por_produtora.keys(), rotation='vertical');
#plt.yscale('log')
plt.show()

# Random Forest de Produtoras

In [ ]:
prod2 = defaultdict(int)
for e in df_mega_clean.index:
    data_dict = ast.literal_eval(df_mega_clean['production_companies'][e])
    for palavras in data_dict:
        produ = palavras['name']
        prod2[produ] += 1
prod2

In [ ]:
ser_produtora = pd.Series(prod2)

In [ ]:
df_filmes_produtoras = pd.DataFrame(index=df_mega_clean.index, columns=ser_produtora.index, dtype=np.uint8)
for e in df_mega_clean.index:
    data_dict = ast.literal_eval(df_mega_clean["production_companies"][e])
    #print(data_dict)
    for palavras in data_dict:
        produ = palavras['name']
        #print(tipo)
        if produ in df_filmes_produtoras.columns:
             df_filmes_produtoras[produ][e] = 1


In [ ]:
df_filmes_produtoras.head()

In [ ]:
df_filmes_produtoras[df_filmes_produtoras != 1] = 0
df_filmes_produtoras.head()

In [ ]:
X3_train, X3_test, Y3_train, Y3_test = train_test_split(df_filmes_produtoras,sim_ou_nao,test_size = 0.3)

model = RandomForestClassifier(n_estimators = 40)
model.fit(X3_train,Y3_train)

In [ ]:
model.score(X3_test,Y3_test)

### Plotando o Histograma das variáveis budget e revenue:

#### Budget:

In [ ]:
df_mega_clean.budget.plot.hist(density = False)

#### Revenue:

In [ ]:
df_mega_clean.revenue.plot.hist(density = False)

### Juntando as duas variáveis num gráfico de dispersão:

#### Gráfico de dispersão juntando o orcçamento e a arrecadação dos filmes:

In [ ]:
df_mega_clean.plot.scatter(x="budget", y = "revenue",  logx=True)

# Random Forest

In [ ]:
df_forest = pd.concat([
    df_filmes_categorias,
    atores_certos.drop(['Fez ou não 150 mi'],axis='columns'),
    df_filmes_produtoras,
    df_mega_clean.budget,

], axis=1)

In [ ]:
X4_train, X4_test, Y4_train, Y4_test = train_test_split(df_forest,sim_ou_nao,test_size = 0.3)

model = RandomForestClassifier(n_estimators = 60)
model.fit(X4_train,Y4_train)

In [ ]:
model.score(X4_test,Y4_test)

In [ ]:
sorted(list(zip(model.feature_importances_, X4_test.columns)), reverse=True)

In [ ]:
model.feature_importances_

# One Hot Encoder